# Representación semántica y búsqueda
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

Como anteriormente se presento, una de las representaciones semánticas más aceptadas son aquellas basadas en word embeddings. Recordando, estas son representaciones vectoriales de cada palabra, donde la semántica se asocia con la estructura en un espacio métrico, i.e., cercano en la métrica significa similar semánticamente.

# Representación semántica de documentos basada en _word embeddings_
Un documento puede verse como una bolsa de palabras, como en el modelado tradicional, pero en lugar de usar los términos como símbolos se pueden usar los vectores semánticos. El como usar esta nube de puntos multidimensional para obtener una representación computacionalmente manejable y a la vez eficaz, es un tema que ha llevado al desarrollo de modelos cada vez más complejos, como pueden ser los grandes modelos de lenguaje. 

La manera forma más directa de definir una representación semántica de documentos es el uso de centroides, esto es, un vector de la misma dimensión que sumariza a un conjunto de vectores usando su media geométrica. De manera más precisa, sea $E$ una matrix $m \times n$ de embeddings del vocabulario, i.e., considerando un vocabulario de tamaño $n$ y dimensión $m$.
Sea $D_{m,\ell}$ la submatriz de $E$ que contiene los $\ell$ vectores del documento $\textsf{doc}$. El prototipo $\vec{d}$ de $D$ esta definido de la siguiente forma:

\begin{equation}
\vec{d} = \frac{1}{\ell} \sum_{i=1}^\ell D_i
\end{equation}


Donde $D_i$ es el i-ésimo vector columna de $D$. Dado que se usa el coseno como similitud, también es factible el cálculo como la suma vectorial normalizada, esto es,

$$ \vec{d} = \frac{\sum_{i=1}^\ell D_i}{\lVert \sum_{i=1}^\ell D_i \rVert}$$

También es posible añadir información local basada en la representación de bolsa de plabras. Por ejemplo, realizar una suma pesada usando el peso de las palabras mediante la frecuencia normalizada de término (TF) o la probabilidad de término $pt$, ver unidad 3 para más información.

$$\vec{d} = \sum_{i=1}^{\ell} \textsf{TF}(t_i, \textsf{doc}) \cdot D_i$$

Donde $t_i$ es el $i$-ésimo término de $\textsf{doc}$. Note que el orden no se captura y solo es necesario para la notación.

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \max \left\{ \sum_{u \in U} \min_{x \in V} d(u, x), \sum_{v \in V} \min_{x \in U} d(v, x) \right\}$$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{TF}$

$$ H^\textsf{TF}_d(U, V) = \max \left\{ \sum_{u \in U} {\textsf{TF}(u, U) \min_{x \in V} d(u, x)}, \sum_{v \in V} {\textsf{TF}(v, V) \min_{x \in U} d(v, x)} \right\} $$

# Notas adicionales
La versión de $H_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:

using Pkg
Pkg.activate(".")
!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.9"),
    PackageSpec(name="TextSearch", version="0.12"),
    PackageSpec(name="Plots"),
    PackageSpec(name="KNearestCenters"),
    PackageSpec(name="HypertextLiteral"),
    PackageSpec(name="CSV"),
    PackageSpec(name="DataFrames"),
    PackageSpec(name="Embeddings", version="0.4")
])

using SimilaritySearch, TextSearch, Plots, KNearestCenters, LinearAlgebra, Embeddings, HypertextLiteral, CSV, DataFrames, Base64, Random
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
]status

      Status `~/IR-2022/Unidades/Project.toml`
  [944b1d66] CodecZlib v0.7.0
  [ac1192a8] HypertextLiteral v0.9.4
  [b20bd276] InvertedFiles v0.4.1 `../../Research/InvertedFiles.jl`
  [682c06a0] JSON v0.21.3
  [4dca28ae] KNearestCenters v0.7.1
  [8ef0a80b] Languages v0.4.3
  [eb30cadb] MLDatasets v0.7.3
  [053f045d] SimilaritySearch v0.9.4 `../../Research/SimilaritySearch.jl`
  [fb8f903a] Snowball v0.1.0
  [2913bbd2] StatsBase v0.33.18
  [7f6f6c8a] TextSearch v0.12.5 `../../Research/TextSearch.jl`


In [3]:
function knn(index, labels, q, k)
    res = KnnResult(k)
    search(index, q, res)
    mode(labels[idview(res)])
end

function mymode(c, labels, f)
    n = length(c)
    empty!(f)
    for id in c
        id == 0 && break  # searchbatch stores zeros at the end of the result when the result set is smaller than the required one
        l = labels[id]
        f[l] = get(f, l, 0) + 1
    end
    
    if length(f) == 0
        rand(labels)
    else
        argmax(last, f) |> first
    end
end

function knn(I, labels)
    f = Dict{eltype(labels), Int}()
    [mymode(c, labels, f) for c in eachcol(I)]
end

function scores(gold, pred)
    s = classification_scores(gold, pred)
    (macrof1=s.macrof1, macrorecall=s.macrorecall, accuracy=s.accuracy)
end

scores (generic function with 1 method)

In [4]:
struct ValidVocabulary{DictType} <: AbstractTokenTransformation
    valid::DictType
end

TextSearch.transform_unigram(S::ValidVocabulary, tok) = haskey(S.valid, tok) ? tok : nothing

function text_model_and_vectors(
        corpus, vocab;
        localweighting=TpWeighting(),
        globalweighting=BinaryGlobalWeighting(),
        nlist=[1],
        qlist=[],
        slist=[],
        group_usr=true,
        group_url=true,
        group_num=true,
        del_diac=true,
        lc=true
    )

    voc = Vocabulary(length(corpus))
    for v in vocab
       push!(voc, v)
    end
    
    tt = ValidVocabulary(voc.token2id)
    textconfig = TextConfig(; group_usr, group_url, del_diac, lc, group_num, nlist, qlist, slist, tt)
    tokenize_and_append!(voc, textconfig, corpus)
    model = VectorModel(globalweighting, localweighting, voc)
    (; textconfig, model, voc)
end


text_model_and_vectors (generic function with 1 method)

In [5]:
function embeddings()
    embfile = "../data/MX.vec"
    !isfile(embfile) && download("http://geo.ingeotec.mx/~sadit/regional-spanish-models/MX.vec", embfile)
    emb = load_embeddings(FastText_Text, embfile)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end
    
    (; X=emb.embeddings, vocab=emb.vocab)
end

function vectorize_as_prototype!(c, text, E, T)
    x = vectorize(T.model, T.textconfig, text; normalize=false)
    if length(x) == 1 && haskey(x, 0)
        #@warn "empty vector $(Int(i)) selecting a random vector for it " # $(corpus[i])
        rand!(c)
    else
        for (id, weight) in x
            id == 0 && continue
             c .= c .+ weight .* view(E.X, :, id)
        end
    end
    
    normalize!(c)
end

function vectorize_corpus_as_prototypes(corpus, E, T)
    dim = size(E.X, 1)
    n = length(corpus)
    C = zeros(Float32, dim, n)
    # Threads.@threads
    for i in 1:n
        vectorize_as_prototype!(view(C, :, i), corpus[i], E, T)
    end
    
    C
end

function create_index(db)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(0.9)))
    optimize!(index, MinRecall(0.9))
    index
end

create_index (generic function with 1 method)

In [6]:
E = embeddings()
D = CSV.read("../data/spanish-news-in-twitter.csv", DataFrame; delim=',')
D[:, :text] = String.(base64decode.(D.text))
dropmissing!(D)
T = text_model_and_vectors(D.text, E.vocab)

(textconfig = TextConfig(true, false, false, true, true, true, false, true, Int8[], Int8[1], Skipgram[], ValidVocabulary{Dict{String, UInt32}}(Dict{String, UInt32}("alaann" => 0x0005f201, "coprolito" => 0x0005f0e5, "alissonrojas" => 0x00018ff1, "copeamos" => 0x0004490d, "inhiben" => 0x00015522, "btardes" => 0x00028895, "guardalo" => 0x00004561, "kokis" => 0x00030576, "sentando" => 0x000087bd, "salvaguarde" => 0x00028f60…))), model = {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=438136, train-n=41998, maxoccs=59524}, voc = Vocabulary(["</s>", "_usr", "que", "de", ",", ".", "y", "a", "la", "no"  …  "tannehil", "movelike", "peliblanca", "cristianzav", "rqq", "preguntartee", "sutedes", "venciendolo", "eviscerado", "fectivo"], Int32[0, 25844, 20315, 59524, 31605, 21433, 17403, 20211, 34402, 6002  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Int32[0, 25844, 20315, 59524, 31605, 21433, 17403, 20211, 34402, 6002  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Float32[

In [7]:
Threads.nthreads()

64

In [8]:
@time C = vectorize_corpus_as_prototypes(D.text, E, T)

  2.292803 seconds (4.46 M allocations: 389.881 MiB, 23.21% compilation time)


300×41998 Matrix{Float32}:
 -0.0339036   -0.0428609   -0.0670109    …  -0.0374637   -0.0475409
  0.0265874    0.0223019    0.00263203       0.0131571    0.018139
 -0.0370614    0.00167701  -0.0378757       -0.0102305    0.0157182
  0.0181526    0.0105444   -0.0240541       -0.0646235    0.0214401
  0.0134366    0.0189434    0.0366603       -0.0527604   -0.00279745
 -0.0213551   -0.0188397    0.0460011    …   0.0460707    0.0460432
  0.0239996   -0.018044    -0.000361117     -0.00433237   0.0459184
  0.0195599    0.0119689    0.00421374       0.0138044   -0.0359088
  0.0655299    0.0465704    0.110764         0.0814923    0.0444924
 -0.00386314   0.0205548   -0.0155296        0.0353385   -0.00603773
  0.0261242    0.0328563    0.0815982    …   0.0578312    0.0652253
 -0.0106883    0.0192735    0.0482066        0.00723559   0.106062
  0.0287057    0.0271132    0.0213498        0.0690113    0.0219228
  ⋮                                      ⋱               
 -0.0379055   -0.0606254   -0.0

### Se crea el índice métrico

In [9]:
@time index = create_index(MatrixDatabase(C))

  7.269850 seconds (12.16 M allocations: 1.251 GiB, 9.08% gc time, 83.60% compilation time)


SearchGraph{NormalizedCosineDistance, MatrixDatabase{Matrix{Float32}}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: MatrixDatabase{Matrix{Float32}}
  links: Array{Vector{Int32}}((41998,))
  locks: Array{Base.Threads.SpinLock}((41998,))
  hints: Array{Int32}((104,)) Int32[5, 19, 20, 153, 263, 289, 343, 357, 399, 429  …  2985, 3015, 3035, 3108, 3114, 3126, 3147, 3148, 3182, 3213]
  search_algo: BeamSearch
  verbose: Bool false


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad (~42k consultas)

In [16]:
let k = 32
    t1 = @elapsed I, _ = allknn(index, k)
    # WARNING: Don't run the following line, it takes too much time
    t2 = @elapsed gI, _ = allknn(ExhaustiveSearch(; db=index.db, dist=index.dist), k)
    r = macrorecall(gI, I)
    n = size(I, 2)
    labels = String.(D.screen_name)
    s1 = scores(labels, knn(I, labels))
    s2 = scores(labels, knn(gI, labels))
    
    @htl """
    <div>macro-recall: $r, n: $n</div>
    <div>searchgraph search time: $t1, scores: $s1</div>
    <div>brute force search time: $t2, scores: $s2</div>
    """
end

┌ Info: precision is zero for label 'fabriziomejia'; #classes=67 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'lydiacachosi'; #classes=67 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'brozoxmiswebs'; #classes=67 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'patriciajaniot'; #classes=67 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'laoctava_tv'; #classes=67 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'jrisco'; #classes=67 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'JorgeGCastaneda'; #classes=67 
└ @ KNearestCente

<div>macro-recall: 0.9062001464355446, n: 41998</div>
<div>searchgraph search time: 0.554506423, scores: <span class="Core-NamedTuple">(macrof1 = 0.18751074131150597, macrorecall = 0.17850393882051546, accuracy = 0.29658555169293777)</span></div>
<div>brute force search time: 6.619088688, scores: <span class="Core-NamedTuple">(macrof1 = 0.2033021713299911, macrorecall = 0.18992704979181993, accuracy = 0.3124434496880804)</span></div>

### Búsqueda y presentación de los resultados

In [11]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = zeros(Float32, size(E.X, 1))
    vectorize_as_prototype!(q, qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.screen_name[id])</td><td> $(D.text[id])</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [17]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  0.000193 seconds


1,26294,0.113,Radio_Formula,El presidente Andrés Manuel López Obrador exhibió el supuesto sueldo de Loret de Mola... con un documento con errores ortográficos. https://t.co/nx4bDFfw8p
2,40773,0.114,El_Universal_Mx,Sigue lo más relevante de la conferencia del presidente Andrés Manuel López Obrador ⬇️ https://t.co/pA4oo1iCbU
3,41633,0.116,PaolaRojas,"RT @nmas: El presidente de México, Andrés Manuel López Obrador, habló en su conferencia mañanera sobre el presunto cobro de derecho de piso…"
4,21912,0.122,CNNEE,Así pensó un plan B el presidente de México Andrés Manuel López Obrador para organizar la consulta de revocación de mandato. https://t.co/GaAeHdjhAG
5,40400,0.125,RicardoAlemanMx,RT @ContrapesoC: El expresidente Vicente Fox se lanzó contra el presidente Andrés Manuel López Obrador por sus declaraciones y por la inseg…
6,8834,0.126,azucenau,Informa @RicardoAnayaC que ha sido citado en el Reclusorio Norte por lo que acusa persecución de parte del presidente Andrés Manuel López Obrador
7,24541,0.127,azucenau,"#ALMOMENTO | Por segundo día consecutivo, el presidente Andrés Manuel López Obrador salió esta tarde de Palacio Nacional. https://t.co/tdBqBYhhuc"


  0.000467 seconds


1,24023,0.245,LaRazon_mx,#SSPC: Red de tráfico de migrantes es ubicada en #Guanajuato y #Tamaulipas https://t.co/2TdiV3G2Zf https://t.co/hctIUWIx7G
2,27494,0.267,CiroGomezL,"Mil 500 elementos de la @SEMAR_mx llegaron al AICM para tomar el control de la seguridad. Los marinos realizarán tareas de vigilancia, inspección, apoyo y control para evitar el tráfico de armas, drogas, divisas, mercancía ilegal y hasta el tráfico de personas: https://t.co/85QYwB26l8"
3,29416,0.28,El_Universal_Mx,RT @ElUniversal_SLP: #Entérate ⛔️ Los trabajos de repavimentación de la carretera 57 provocan filas kilométricas de vehículos desde tempran…
4,29985,0.285,EFEnoticias,"Un centenar de profesores mantienen cortado esta mañana el tráfico en la Ronda Litoral, a la altura de la Vila Olímpica de Barcelona, lo que está causando grandes retenciones de tráfico en esta vía en el tercer día de huelga de la enseñanza. https://t.co/jIsziPxkn7"
5,41311,0.286,Reforma,#Entérate Intentos de asalto y poca infraestructura carretera en el Estado de México obstaculiza el reparto de ayuda del Banco de Alimentos Poniente https://t.co/MPpkMmpRX8
6,36859,0.287,El_Universal_Mx,"De acuerdo con datos de la Administración de Control de Drogas (DEA) de Estados Unidos, en la versión original, el fentanilo es considerado de 30 a 50 veces más potente que la heroína https://t.co/9mWdq9W8PA https://t.co/bqchRWkYvZ"
7,32937,0.288,Radio_Formula,"Un taxi fue de los vehículos afectados por el accidente de tránsito en Tlalpan, Ciudad de México. https://t.co/pJaNrYlRRH"


  0.000613 seconds


1,37493,0.462,abc_es,Primeros datos de la eficacia de dos antivirales frente al virus de la viruela del simio https://t.co/TdBHvIIT3w
2,31766,0.468,AristeguiOnline,Molnupiravir elimina el virus SARS-CoV-2 activamente infeccioso al tercer día de comenzar la terapia: estudio https://t.co/93yBkWlj3G
3,24558,0.476,El_Universal_Mx,Es el primer deceso registrado en la entidad por esa variante del virus SARS-CoV-2 https://t.co/3FlnzSNpbk
4,16409,0.477,bbcmundo,El alentador efecto de la vacuna contra el virus del papiloma humano (VPH) en la reducción del cáncer de cuello uterino https://t.co/rQV9L9Q9F4
5,38127,0.482,SinEmbargoMX,Fármacos contra enfermedad inflamatoria intestinal protegerían de la COVID grave https://t.co/b7yXzIGahI https://t.co/nGZVBpX9Xi
6,38229,0.492,Adela_Micha,RT @LaSagaOficial: El infectólogo Carlos Aguirre habla sobre los virus que se han asociado a la hepatitis aguda infantil “Uno se llama ad…
7,11175,0.495,PublimetroMX,Desde 2018 se han detectado tres brotes del virus Nipah; 70% de los contagiados mueren https://t.co/ck7YVHYHtV


In [13]:

display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")
for i in 1:3
    for qid in rand(1:length(D.text))
        search_and_display(index, D.text[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.000355 seconds


1,21443,-0.0,AdriDelgadoRuiz,"“El presidente @lopezobrador_ ha marcado muy claro su desdén por la intelectualidad, pero por la que no lo acompaña”: Revive la #Entrevista al periodista @ricardomraphael #ElDedoEnLaLlaga @heraldodemexico #HeraldoTV https://t.co/hVsZwrZYrj https://t.co/uc8SbYsqiB"
2,3176,0.06,MarioBeteta,"@carloselizondom @Radio_Formula El columnista @carloselizondom se dice preocupado por la hegemonía que ha ganado la CFE con el actual gobierno, pues no la ve ""capaz de atender todas las necesidades del suministro de la energía eléctrica""."
3,21871,0.061,RicardoAlemanMx,"No, querido @Amadoelquelolea ! El tal @DrLorenzoMeyer ya es parte de la cloaca en que @lopezobrador_ convirtió la politica y el poder! Y es que, como dijo un clásico: ""lo doctor no quita lo pendejo!"" Abrazo!!! https://t.co/wyPW77uzih"
4,19427,0.061,Radio_Formula,#Opinión | @RicardoMonrealA está decidido y el presidente lo sabe. La Presidencia la decidirá el mejor candidato y en eso Monreal los aventaja por ser el único que va por la libre. Escribe @lopezdoriga. https://t.co/4tN9JMYdrl https://t.co/rSwnie43FI
5,35822,0.061,SinEmbargoMX,"RT @SinEmbargoTV: #Opinión | ""La oposición no levanta"" por @alvaro_delgado “El problema más serio que enfrenta la oposición, sin embargo,…"
6,22363,0.062,Milenio,"#PolíticaCero | “Peor está el ChikiliQuadri que, además del comunismo, también le declaró la guerra al reguetón, lo cual solo ha conseguido que repunte el perreo” 🎙 Lee y escucha la columna de @JairoCalixto https://t.co/Ud7jyNsJ7W https://t.co/kHz93VdTd8"
7,34833,0.062,RicardoAlemanMx,"Imaginan lo que habría dicho @lopezobrador_ si @EPN , @FelipeCalderon o @VicenteFoxQue hubiesen dicho lo que hoy dijo 👇 el dictador López? Pero claro, la mafia lopista calla! Vamos a seguir ganando por “paliza” en 2024: AMLO https://t.co/eLzIqHRP2q vía @ricardoalemanmx"


  0.000247 seconds


1,31049,0.0,El_Universal_Mx,"RT @de_ultima: #KristenStewart es llamada la rebelión de la moda, llega a la red carpet de los #Oscars en pantalón corto de @CHANEL #Oscars…"
2,10004,0.057,el_pais,RT @elpaissemanal: Es la reinventora de la novela de amor decimonónica en la era de la hiperconectividad. La escritora superventas de los m…
3,16785,0.066,teleSURtv,"La @UNESCO_es anunció la #incorporación de la ciudad oriental #SantiagodeCuba a la Red de #CiudadesCreativas, por su #aportemusical, una de las categorías reconocidas👉https://t.co/XWdapowvLi"
4,7748,0.066,azucenau,RT @FiscaliaCDMX: El día de hoy se hace historia; la justicia por encima de la política. Gracias a la decisión de las y los @Mx_Diputados p…
5,32615,0.068,El_Universal_Mx,"""Es estar sumando a la polarización"", considera @LuisCardenasMx sobre el mensaje de la playera de @mario_delgado en conferencia por la #RevocaciónDeMandato https://t.co/nqVxwzyxcN https://t.co/uShp8tRTOU"
6,38782,0.07,epigmenioibarra,"RT @AliciaMonicaOT: Es tiempo de forma parte de la historia. Vamos que no quede duda, a votar en favor de la transformación"
7,22733,0.07,julioastillero,"RT @reforma2006: no se pierdan a las 7 de la noche por @Canal22 la repetición de la ""mesa del mas allá"" con @anafrancismor ,@HoracioFranco…"


  0.000307 seconds


1,5056,0.0,El_Universal_Mx,RT @ElUniversalOax: #Entérate 👨🏻‍💻 Un doodle de Google es una alteración temporal del logotipo en la página principal de Google que pretend…
2,26658,0.113,galvanochoa,RT @pictoline: Mucho ojo 👀: la historia de cómo un guardia arruinó una pintura de la era soviética... en su primer día de trabajo 🖋🙃 ¿Cuál…
3,7284,0.118,El_Universal_Mx,RT @ElUniversal_SLP: #Entérate 🚦 El cambio de semaforización se debe a una alta incidencia de transmisión del virus SARS-CoV-2 en la entida…
4,10189,0.119,El_Universal_Mx,RT @ElUniversal_SLP: #LoMásLeído 🌎 Google Earth Engine creó un timelapse de la capital potosina y su Zona Metropolitana https://t.co/El5Sf…
5,4141,0.119,El_Universal_Mx,RT @UniversalPuebla: 📶 No es tu internet. Falla la página de la @SEPGobPue para consultar el modelo híbrido de clases. https://t.co/T1Xs13…
6,19087,0.121,El_Universal_Mx,"RT @De10mx: #Covid 😷🇲🇽 Un especialista de la UNAM afirmó que es “prácticamente imposible” evitar que ómicron, la nueva variante de Covid, l…"
7,14057,0.121,teleSURtv,"Linkedin: Envía datos de #telemetría a #Microsoft, los cuales se difunden con cualquier proveedor de análisis con el que la 'app' se haya conectado📵 Mensajería de Google: Comparte marcas de tiempo de cada interacción del usuario con Google Analytics, además de cada mensaje📱 https://t.co/n1d7a2dkW6"


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
